In [5]:
from gurobipy import Model, GRB, quicksum, LinExpr

In [2]:
class clustering_model(object):
    def __init__(self, n_vertices, edges, constraints, k, gamma):
        self.check_graph(n_vertices, edges)

        model = Model('graph_clustering')
        
        mvars = []
        for i in range(k):
            cvars = []
            for j in range(n_vertices):
                v = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
                cvars.append(v)
            mvars.append(cvars)
            
        # constraint: each vertex in exactly one cluster
        for v in range(n_vertices):
            model.addConstr(quicksum([mvars[i][v] for i in range(k)]), GRB.EQUAL, 1)
        
        
        obj_expr = LinExpr()
        
        # indicators for violation of cl constraints
        for (u, v, w) in constraints:
            for i in range(k):
                y = model.addVar(lb=0.0, ub=0.0, vtype=GRB.BINARY)
                model.addConstr(y >= mvars[i][u] + mvars[i][v] - 1)
                obj_expr.add(y, -w * gamma)
        
        # size of smallest cluster 
        s = model.addVar(lb=0.0, ub=n_vertices, vtype=GRB.INTEGER)
        for i in range(k):
            model.addConstr(s <= quicksum([mvars[i][v] for v in range(n_vertices)]))
        obj_expr.add(s)
        
        model.setObjective(obj_expr, GRB.MAXIMIZE)
        model.Params.PreCrush = 1
        
        model._cut_finder = Cut_Finder(n_vertices, edges)
        model._vars = mvars
        model._k = k
        model._relobj = None
        model._impcounter = 0
        
        self.model = model
               
    def check_graph(self, n_vertices, edges):
        vertices = [i for (i, _) in edges]
        vertices += [i for (_, i) in edges]
        assert(sorted(vertices) == list(range(n_vertices))
        for u, v in edges:
            assert(u < v)
            assert(u < n_vertices)
    
    def solve(self):
        self.model.optimize(mincut_callback)
        print(self.model.ObjVal)
    